# 線形モデル選択と正則化

ここでは線形モデルを改良する別の方法を考える。線形モデルには推論の点で明確な長所があり、現実の問題を扱う際には非線形法と比較して消して劣ることはない。しかし、他の方法の方が予測精度が高く、そして、モデルの解釈が容易になる状況がある。
- 予測精度：応答変数と予測変数の間の真の関係が近似的に線形であるならば、最小2乗法による推定値のバイアスは小さい。$n > p$の場合は分散も小さくなる傾向にある。ゆえにテストデータにおいて良い性能を発揮する。しかし、もし$n$が$p$よりあまり大きくない場合、最小2乗法による予測はばらつきが生じ、過学習となり、訓練に使われなかったデータにおける予測精度は落ちることになる。また$p > n$の場合、最小2乗法では回帰係数が一意に定まらない（使えなくなる）。
- モデルの解釈：予測変数のうちいくつか、または多くが実は応答変数と関係がないということはよくある。このような無意味な変数を入れてしまうことでモデルは必要以上に複雑になってしまう。これら不必要な変数を取り除く、つまり、該当する係数を0にすることにより、より解釈が容易なモデルにすることができる。最小2乗法では係数が正確に0になることはそれほど起こらない。

最小2乗法を線形モデル$ Y = \beta_0 + \beta_1 X_1 + \cdots + \beta_p X_p + \varepsilon $に当てはめることに代わる方法はいくつかあるが、ここでは**部分集合選択**と**縮小推定**について論じる。

## 部分集合選択

### 最良部分集合選択

step1: $\mathcal{M}_0$を予測変数を持たないヌルモデルとする。このモデルは予測値として単に標本平均を用いる。

step2: $k=1,\ldots,p$について：

(a)$k$個の予測変数をもつモデル$\binom pk$個全てに回帰を当てはめる。

(b)$\binom pk$個のモデル全てからベストなものを選ぶ。これを$\mathcal{M}_k$とする。ベストなモデルとはRSSが最小となるもの、または$R^2$が最大となるものである。

step3: $\mathcal{M}_0, \ldots, \mathcal{M}_p$のうち、ただ一つのベストなモデルを選ぶ。判断規準は交差検証予測誤差、Cp(AIC)、BIC、自由度調整済み決定係数$R^2$などである。

最良部分集合選択には計算量の面で問題がある。一般的に$p$個の予測変数があれば考慮するモデルの数は$2^p$個となる。$p=10$であればおよそ1000個のモデルを検討するし、$20$ならば100万個以上のモデルを検討しなければならない。そのため最良部分選択集合選択は最新のコンピュータをもってしても$p=40$以上の場合には計算量的に困難である。

### ステップワイズ法：変数増加法と変数減少法

#### 変数増加法

step1: $\mathcal{M}_0$を予測変数を持たないヌルモデルとする。

step2: $k=1,\ldots,p-1$について以下を繰り返す：

(a) $\mathrm{M}_k$の予測変数に含まれない変数のうち、どれか一つを加えることによって構成される$p-k$個のモデルを考える。

(b) $p - k$個のモデルのうち、ベストなものを選び、これを$\mathcal{M}_{k+1}$とする。ここにベストとは、RSSが最小または$R^2$が最大であることとする。 

step3: 交差検証予測誤差、Cp（AIC）、BIC、自由度調整済み$R^2$などにより$\mathcal{M}_0, \ldots, \mathcal{M}_p$個の中からベストなモデルを選ぶ。


- 変数増加法は最良部分集合選択に代わる方法として計算量的に効率の良い方法である。最良部分選択集合では$2^p$個のモデルを当てはめることになるのに対し、変数増加法はまず最小にヌルモデル、そして$k$回目（$k=0, \ldots, p-1$）の繰り返しループで$p-k$個のモデルを当てはめる。これによっって検討するモデルの総数は$1 + \sum_{k=0}^{p-1}(p - k) = 1 + p(p+1)/2$個となる。これは大きな違いである（例：$p=20$のとき、最良部分集合選択では1048576個のモデルを検討する必要があるが、変数増加法ならば211個のみでよい）。
- 変数増加法は計算量の面で最良部分集合選択に勝っており、実用上良い性能を示すが、$p$個の変数で考えうる$2^p$個全てのモデルのうちベストなものを発見できるという保証はない。例として3個の予測変数（p=3）をもつデータセットを考える。1変数でのベストなモデルは$X_1$を含み、2変数でのベストなモデルは$X_2$と$X_3$を使ったものであるとする。この場合、変数増加法では2変数のベストモデルを見つけることができない。$\mathcal{M}_1$は$X_1$を含むため、$\mathcal{M}_2$は$X_1$ともう一つ別の変数を含むことになるからである。
- 変数増加法は高次元（$n < p$）の場合にも適用することができる。しかしこの場合、モデルは$\mathcal{M}_0, \ldots, \mathcal{M}_{n-1}$までとなる。これは$p \ge n$の場合、モデルを当てはめるのに使われる最小2乗法の解が一意に定まらないからである。

#### 変数減少法

step1: $p$個すべての予測変数を含むフルモデルを$\mathcal{M}_p$とする。

step2: $k=p, p-1, \ldots, 1$について以下のループを繰り返す；

(a) $\mathcal{M}_k$から予測変数を1つだけ除いてできる$k$個のモデルを考える。これらは$k-1$個の予測変数を持つ。

(b) これら$k$個のモデルのうちベストなモデルを選び、これを$\mathcal{M}_{k-1}$とする。ここに、ベストとはRSSが最小なもの、または$R^2$が最大なものとする。

step3: $\mathcal{M}_0, \ldots, \mathcal{p}$のうち、ベストなモデルを交差検証予測誤差、Cp（AIC）、BIC、自由度調整済み$R^2$などにより選ぶ。

- 変数増加法と同様、変数減少法は最良部分集合選択に代わる効率的な方法である。変数増加法と同様に変数減少法が探索するのは$1 + p(p+1)/2$個のモデルのみである。したがって最良部分集合選択が適用できないほど$p$が大きなケースにも適用することができる。
- 変数増加法と同じく、$p$個の予測変数のすべての部分集合でベストなモデルを発見できるという保証はない。
- 変数減少法を行うには、（フルモデルをあてはめるため）サンプルサイズ$n$が変数の数$p$よりも大きいという条件を必要とする。これに対し、変数増加法は、$n < p$の場合にも使うことができる。したがって$p$が非常に大きい場合には唯一の実行可能な方法といえる。

上記3つの方法を実装するには、モデルを評価する方法が必要である。実は、すべての予測変数を含むモデルが常にRSSを最小、$R^2$を最大とする。なぜならこれらの量は訓練誤差に関係しているからである。しかし本来は訓練誤差ではなく、テスト誤差を最小にするようなモデルを選びたい。訓練誤差をテスト誤差の推定値として使用することは、訓練MSEが小さければテストMSEが小さいという保証がないため、あまりよくない。したがってRSSや$R^2$は予測変数の数が異なるモデルを比較して最適なものを選ぶ際には適していない。

テスト誤差について最適なモデルを選ぶため、テスト誤差を推定する必要があるが、良く用いられる手法には2種類ある：
    1. 訓練誤差を過学習によるバイアスを考慮して調整することにより、テスト誤差を間接的に推定する。
    2. ホールドアウト検証や交差検証法により直接的にテスト誤差を推定する。

### $C_p$, AIC, BIC, 自由度調整済み$R^2$

モデルの予測変数の数を増やせば訓練誤差は減少するが、テスト誤差は減少するとは限らない。よって訓練RSSや訓練$R^2$は変数の数が異なるモデルの中から最適なモデルを選ぶ際には使うことはできない。しかし、モデルの予測変数の数に応じて、訓練誤差を調整する方法はいくつもある。

$d$個の予測変数をもつ最小2乗モデルにおいて、$C_p$によるテストMSE推定値は次で計算される：

\begin{equation}
    C_p = \frac{1}{n}(\mathrm{RSS} + 2 d \hat{\sigma}^2)
\end{equation}

ここで、$\hat{\sigma}^2$は誤差$\epsilon$の分散の推定量である。$\hat{\sigma}^2$が$\sigma^2$の推定値としてバイアスのないものであるならば、$C_p$はテストMSEの推定値としてバイアスが無いとことが示される。結果として、テスト誤差の低いモデルで$C_p$統計量は小さい値を示すこととなり、最適なモデルを選ぶ際には$C_p$の値が最小となるものを選べばよい。

AIC規準は最尤法によるさまざまなクラスのモデルの当てはめに対し定義される：

\begin{equation}
\mathrm{AIC} = \frac{1}{n \hat{\sigma}^2}(\mathrm{RSS} + 2 d \hat{\sigma}^2) 
\end{equation}

BICはベイズ統計の見地に立って定義されているが、$C_p$、AICに類似した形になっている。

\begin{equation}
\mathrm{BIC} = \frac{1}{n \hat{\sigma}^2}(\mathrm{RSS} + 2 d \hat{\sigma}^2) 
\end{equation}

$C_p$と同様、テスト誤差が小さいときにBICも小さい値をとる。したがって、一般的にはBICの値が最小になるモデルを選べばよい。

自由度調整済み$R^2$も変数の数が異なるモデル族の中から選ぶ際によく使われるアプローチである。通常の$R^2$は$R^2 = 1 - \mathrm{RSS}/\mathrm{TSS}(\mathrm{TSS} := \sum (y_i - \bar{y})^2)$であるが、自由度調整済み$R^2$は次のように計算される：

\begin{equation}
    \text{自由度調整済み}R^2 = 1 - \frac{\mathrm{RSS}/(n-d-1)}{\mathrm{TSS}/(n-1)} 
\end{equation}

テスト誤差が小さいモデルで$C_p$、AIC、BICが小さい値を示すのに対し、自由度調整済み$R^2$は大きい値となる。直観的には、自由度調整済み$R^2$はモデルに含まれるべき変数を含んだうえでそれ以上他のノイズ変数をモデルに加えると、RSSがわずかだけ減少する。ノイズ変数を加えればもちろん$d$は増加し、$\frac{RSS}{n-d-1}$は増加する。結果として自由度調整済み$R^2$は減少する。$R^2$と異なり、自由度調整済み$R^2$はモデルに不必要な変数を含めるとその対価を支払わなければならないのである。

### ホールドアウト検証と交差検証

ホールドアウト検証や交差検証法を使って直接テスト誤差を推定することもできる。対象となるモデルのホールドアウト検証誤差や交差検証誤差を計算し、推定誤差が最小となるようなモデルを選べばよい。この手法は直接テスト誤差を推定しており、また真のモデルについての仮定がより少ないという点でAIC、BIC、$C_p$、自由度調整済み$R^2$より優れているといえる。モデルの自由度（p）の決定が難しい場合や誤差の分散$\sigma^2$の推定が難しい場合などでさえ用いることができる。従来$p$が大きい場合や$n$が大きい場合には交差検証に必要な計算量が莫大であることから、モデル選択にはAIC、BIC、$C_p$、自由度調整済み$R^2$がよく使われた。しかし、現在ではコンピュータの高速化により、交差検証をするのに必要な計算量はもはや問題になることはない。したがって、交差検証は検討される多くのモデルのうち最適なものを選ぶ際のより魅力的なアプローチである。

## 縮小推定

**部分集合選択**は、予測変数の部分集合を含む線形モデルに最小2乗法を当てはめた。それに代わる方法として、p個全ての変数をモデルに含み、係数の推定値を制約、あるいは正則化する仕組みを用いてモデルを当てはめる。つまり、係数を0に近づけることを考える。これにより分散がかなり減少するのである。回帰係数を縮小して0に近い値にする（または0にする）方法のうち、最もよく知られているのがリッジ回帰とLassoである。

### リッジ回帰

最小2乗法では以下を最小化することで$\beta_0, \ldots, \beta_p$を推定していた：

\begin{equation}
    \mathrm{RSS} = \sum_{i=1}^{n} \left(y_i - \beta_0 - \sum_{j=1}^{p}\beta_j x_{ij}\right)^2
\end{equation}

リッジ回帰は最小2乗法と類似しているが、係数を推定するとき少し異なる関数を最小化することになる：

\begin{equation}
    \sum_{i=1}^{n} \left(y_i - \beta_0 - \sum_{j=1}^{p}\beta_j x_{ij}\right)^2 + \lambda \sum_{j=1}^p \beta_j^2 = \mathrm{RSS} + \lambda \sum_{j=1}^{p} \beta_j^2 \tag{6.5}
\end{equation}

ここで$\lambda \ge 0$はチューニングパラメータであり、別途決定する必要がある。式(6.5)には2つの異なる基準のトレードオフがある。最小2乗法と同様、リッジ回帰はRSSを最小化することによりデータによく当てはまる係数を推定する。しかし、$\beta_1, \ldots, \beta_p$が0に近いと、罰則項と呼ばれる第2項$\lambda \sum_j \beta_j^2$も小さいため、$\beta_j$の推定値を0に近づける（縮小する）効果がある。チューニングパラメータ$\lambda$はこれらの2項が回帰係数の推定値に与える影響を制御するためにある。$\lambda = 0$のとき、罰則項は何の影響ももたらさないため、リッジ回帰は最小2乗法となる。しかし$\lambda \to \infty$とすると、罰則項の影響が大きくなり、リッジ回帰係数は0に近づいていく。

標準の最小2乗法による回帰係数の推定値は、スケールの変更に対して等価である。つまり、$X_j$を定数$c$倍すると係数は単に$1/c$倍になるだけである。別の言い方をすれば、$j$番目の予測変数がどんなスケールであろうが、$X_j \hat{\beta}_i$は不変である。それに対し、リッジ回帰係数は予測変数を定数倍した際には大きく変化する。例えば単位をドルとする変数``income``を考える。この変数の単位として千ドルを使うこともよくあるが、その場合その変数の値は以前の値と1000倍異なる。ここで、式（6.5）のリッジ回帰には係数の平方和の項があるため、このようなスケールの変化は単に``income``のリッジ回帰係数で1000倍を考慮すればよいというわけにはいかない。すなわち、$X_j \hat{\beta_{j, \lambda}^R}$は$\lambda$の値だけでなく、$j$番目の予測変数のスケールにも依存しているのである。実際のところ、$X_j \hat{\beta_{j, \lambda}^R}$の値は他の予測変数のスケールにも依存しているかもしれない。したがって、すべての変数を同じスケールにするように予測変数を

\begin{equation}
    \tilde{x}_{i, j} = \frac{x_{ij}}{\sqrt{\frac{1}{n}\sum_{i=1}^n (x_{ij} - \bar{x}_j)^2}} \tag{6.6}
\end{equation}

により標準化した上で、リッジ回帰を当てはめるのが最も適切である。式(6.6)の分母は、j番目の予測変数の標準偏差の推定値である。結果として、標準化後は、すべての変数の標準偏差は1になる。最終的な結果は予測変数がどのスケールで測られているかに依存しない。

#### リッジ回帰が最小2乗法よりも優れている理由と欠点

リッジ回帰が最小2乗法より優れている理由は、バイアスと分散のトレードオフにある。$\lambda$が増加するにしたがって、リッジ回帰の柔軟さは減少し、分散は減少するが、バイアスは増加する。

$p=45, n=50$のときのあるシミュレーションデータでは、$\lambda$を適切に選択することで（テキストでは$\lambda=30$付近）、テストMSEを$\lambda=0$のときよりも小さくすることができる。

一般的に、予測変数と応答変数の関係が線形に近いとき、最小2乗法による回帰係数の推定値はバイアスは小さいが分散は大きいかもしれない。これは訓練データにおける小さな変化が最小2乗法が推定する係数に大きな影響をもたらすということである。特に、$p$が$n$と同程度に大きい場合（テキストでは$p=45, n=50$）最小2乗法による回帰係数の推定値には非常に大きなばらつきがある。また、もし$p > n$であれば、最小2乗法による回帰係数の推定値は一意に定まらない。しかし、その場合でもリッジ回帰はバイアスが少し増える代わりに分散を大きく減少させるので、いい結果が得られる。したがって、リッジ回帰は最小2乗法による回帰係数の推定値の分散が大きいときに最も適している手法である。

リッジ回帰は$2^p$個のモデルの探索を必要とする最良部分集合選択において計算量の点で大きな利点をもつ。pの値があまり大きくない場合でさえもこのような探索は計算量が非常に多く、実用的でないこともある。これに対し、どのような$\lambda$の値についても、リッジ回帰はただ1つのモデルを当てはめる。その当てはめの手順の実行は極めて高速であり、式（6.5）をすべての$\lambda$について同時に解くのにかかる時間は最小2乗法を1つのモデルに当てはめるのにかかる時間とほぼ同じであることが知られている。

一方リッジ回帰には明らかな欠点もある。すべての変数の部分集合を使ったモデルを検討する最良部分集合選択、変数増加法、変数減少法などと異なり、リッジ回帰は$p$個全ての予測変数を含むモデルを扱う。式（6.5）の罰則項は係数を0に向かって縮小はするが、（$\lambda \to \infty$としなければ）正確に0にするわけではない。これ自体は予測精度に関して問題がないかもしれないが、変数の数$p$が大きい場合においてモデルの解釈を困難にする。

### Lasso

Lassoは比較的近年になって使われるようになった手法であり、リッジ回帰の解釈性の欠点を克服しようとするものである。Lasso係数$\hat{\beta}_\lambda^L$は以下の量を最小化する：

\begin{equation}
    \sum_{i=1}^{n} \left(y_i - \beta_0 - \sum_{j=1}^{p}\beta_j x_{ij}\right)^2 + \lambda \sum_{j=1}^p |\beta_j| = \mathrm{RSS} + \lambda \sum_{j=1}^{p} |\beta_j| \tag{6.7}
\end{equation}

式（6.7）は式（6.5）と同様の形式をもつが、唯一の違いは罰則項が$\lambda \sum_{j=1}^{p} |\beta_j|$となっている（$l_1$ノルム正則化項を使っている）ことである。

リッジ回帰と同様、lassoも推定係数を0に向かって縮小するが、lassoの場合は、チューニングパラメータ$\lambda$が十分大きいときには$l_1$ノルム罰則項がいくつかの係数を正確に0にする効果をもつ。したがって、最良部分集合選択と同様、lassoは変数選択を行う。結果的にlassoによって作られたモデルは一般的にリッジ回帰によるモデルよりも極めて解釈が容易である。

#### Lassoとリッジ回帰の比較

Lassoはリッジ回帰に対して大きな長所があることは明白である。lassoは予測変数の一部のみを使い、より単純で解釈の容易なモデルを作るからである。どちらの方が予測精度が高いのかについては、lassoとリッジ回帰のどちらかが一貫して他方よりも良いということはない（実際、テキストp208, 209では45個の予測変数を使いそれぞれの性能が良い状況があることが示されている；全てが応答変数に関係するようにシミュレーションデータを作成した場合つまり$\beta_1, \ldots, \beta_{45}$いずれも$0$でない場合にはリッジ回帰の方が性能が良いが、2つしか関係しないようにシミュレーションデータを作成するとlassoの方がバイアス、分散、MSEいずれも性能が良い）。一般には、lassoは、少数の予測変数が主で、その他の係数がとても小さい値または0であるような場合に適していると考えられる。リッジ回帰は応答変数が多くの予測変数の関数で、すべての係数が同程度の値の時に適しているであろう。しかし、応答変数に関係している予測変数の数を実際のデータでは事前に知ることはできない。あるデータセットにおいて、どちらの手法が適しているかを決定するには交差検証などを使うとよい。

最小2乗法による推定値の分散が非常に大きい場合には、リッジ回帰と同様lassoもバイアスをわずかに増やす代わりに分散を減少させることができる。結果として予測精度がより高くなる。リッジ回帰と異なる点は、lassoは変数選択を行うので、解釈が容易なモデルが得られるという点である。

リッジ回帰にもlassoにも非常に効率的なアルゴリズムがあり、どちらの場合においてもすべての$\lambda$について回帰係数を求める問題を最小2乗法を一度当てはめるのと同程度の時間で解くことができる。

### チューニングパラメータの選択

リッジ回帰とlassoでは式（6.5）と（6.7）におけるチューニングパラメータ$\lambda$の値を決定する方法が必要とされる。交差検証は、この問題に対処する単純な方法である。各$\lambda$の値について交差検証誤差を計算する。そして、交差検証誤差が最小になるようなチューニングパラメータを選択すればよい。最後に、改めてすべてのデータに最適なチューニングパラメータを使ってモデルを当てはめる。

チューニングパラメータに対する交差検証誤差のグラフにおいて、最小2乗法に比べてわずかに縮小が行われていることが確認できるとき（小さい$\lambda$で「谷」になっているとき；例えばテキストp213では$\lambda=0.5$のとき）や曲線の谷が明確でないとき（$\lambda$の値が広範囲で同程度の誤差になるとき）には単に最小2乗法を使うとよいかもしれない。

$p=45, n=50$という非常に難しい状況のデータであっても、lassoを適用しCVでチューニングパラメータを選択すれば、シグナル変数のみから構成される正しい変数のセットを選択できることがある。

## 高次元の場合に考慮すべき事項

### 高次元データ

回帰や分類のための統計的手法はほとんどの場合、低次元、すなわち、観測データの数$n$が特徴の数$p$よりもかなり大きい場合を想定している。ここでの次元とは$p$の大きさということである。

ここ20年間の技術進歩により、ファイナンス、マーケティング、医療などの様々な分野でデータを集める技術が変革を遂げた。現在では、ほとんど数えきれないほど多くの特徴を集めることが常態化している（つまり$p$が大きい）。$p$は非常に大きいかもしれないが、観測データの数$n$は費用や標本の手に入れやすさ、その他の理由により、限られた数しか手に入らない。以下に2つ例を示す：

1. 血圧を予測するのに、年齢、性別、BMIのみを使うのではなく、予測モデルに一塩基遺伝子多型（SNP: single nucleotide polymorphisms, 比較的よく見られる単独DNAの突然変異）を含めるかもしれない。このような場合$n \approx 200$、また$p \approx 500,000$となる。
2. マーケティングアナリストがオンラインショッピングにおける購買パターンを知りたいという場合に、オンラインユーザがサーチエンジンに入力した単語全てを特徴とすることもできる（これはbag of wordsと呼ばれるモデル）。ある研究者が数百人または数千人のサーチエンジンユーザの同意を得てすべての検索履歴を記録するような場合である。あるユーザについて、p個の単語について検索した（1）または検索しなかった（0）と記録され、大きな2値ベクトルとなる。このとき$n \approx 1000$で$p$はさらに大きい。

観測データの数よりも特徴の数が多いようなデータセットをしばしば高次元という。このような場合、最小2乗法のような古典的アプローチは適していない。高次元データの解析時に起こりうる問題については、$n > p$のときと同様であり、バイアスと分散のトレードオフや過学習などを含んでいる。これらの問題については教師付きの統計的学習を使う際にはいつも考えなければならないが、特徴の数が観測データの数に比べて非常に大きいときは、特に注意が必要である。

### 高次元の場合における問題点

高次元の場合を想定していない統計的手法を使った場合にどのような問題が起きるかを見る。ここでは最小2乗法による回帰を使うが、以下の考え方はロジスティック回帰、線形判別分析、その他の古典的な統計的手法にも当てはまる。

特徴の数が観測データの数$n$と同じくらいか、またはより大きい場合、最小2乗法は使うことはできない（使うべきではない）。理由は簡単である；特徴と応答の間に真に関係があるかないかに関わらず、最小2乗法は残差が0になるような完全な当てはめとなる係数の推定値を与えてしまうのである。

p=1のとき（線形単回帰）で、データの数が20個と2個の場合を考えてみる。20個のデータがある場合、$n > p$であり、最小2乗法による回帰直線はデータに完全に当てはまることはない。回帰直線は20個のデータをなるべく近似しようとする。その一方、データが2個のみである場合、どのようなデータであろうとも、回帰直線はデータに完全に当てはまる。これは問題である。なぜならば、この完全に当てはまった回帰直線はほぼ間違いなく過学習だからである。つまり、高次元の場合において、訓練データに完全に当てはめることができるが、その結果、得られた線形モデルは別のテストデータでは性能がとても悪くなるであろう。したがって、これでは有益なモデルとは言えない。

$C_p$、AIC、BICなどの手法は高次元の状況下においては適切でない。なぜならば、推定値$\hat{\sigma}^2$に問題が生じるからである（例えば$\hat{\sigma}^2 = \frac{\mathrm{RSS}}{n - p - 1}$は0になってしまう）。同様に、高次元においては自由度調整済み$R^2$を使うことにも問題がある。自由度調整済み$R^2$が1となるようなモデルを作ることは簡単になるためである。明らかに高次元の場合により適している他の手法が必要である。 

### 高次元の場合における回帰分析

あまり柔軟ではない最小2乗法によるモデルを当てはめる手法、つまり変数増加法、リッジ回帰、lasso、主成分回帰などが高次元で回帰分析を行う上でとても有利である。要するに、これらの手法は通常の最小2乗法に比べて柔軟でないために、過学習の問題を回避できるのである。

lassoのあるシミュレーションからは、テスト誤差は問題の次元（特徴または予測変数の数）が大きくなるにつれて増加する傾向があることが示せる。もちろん、新たにモデルに加えた特徴が本当に応答変数と関係していればこの限りではない。この原理は次元の呪いと呼ばれる。モデルを当てはめるのに使われる特徴の数を多くすればするほどモデルの精度は必ずしも改善しないのである（テキストでは$p=20$から$p=2000$に増えた際にテストMSEがほぼ2倍になっている）。一般には応答変数と本当に関係しているシグナル変数をモデルに加えればモデルの改善に繋がる。しかし応答変数に関係していないノイズ変数を加えてもモデルを悪化させることにしかならず、結果としてテスト誤差を増加させてしまう。これはノイズ変数が問題の次元を上げるので、過学習のリスクをより悪化させ、その一方でテスト誤差を改善しないからである（訓練データでたまたま関係があるように見えたノイズ変数に非ゼロの係数を割り当ててしまう）。

### 高次元の場合における結果の解釈

lasso、リッジ回帰、その他の回帰を高次元の状況において行うとき、得られた結果を読み取るには細心の注意を払う必要がある。高次元の場合は多重共線性は非常に大きな問題である。どの予測変数が本当に応答変数に関係しているのか（そもそも関係している予測変数があるのか）について正確な答えを知ることができないということである。また、回帰に使うべきベストな係数についても正しい答えは分からない。できることと言えば、応答変数の予測に本当に必要な変数に大きな回帰係数が付されていると願うことだけである。

例えば、500,000個のSNPをもとに血圧を予測したい場合を考える。変数増加法によって、17個のSNPが訓練データにおいて良い精度を示したとする。これらの17個の変数が、モデルに含まれなかった他の変数よりも血圧を予測するのにより優れていると結論付けることは間違っている。選ばれた17個のSNPのモデルと同程度の精度で血圧を予測できる17個のSNPの組み合わせは他にも多くあることだろう。独立したデータセットが別にあり、そのデータで変数増加法を使えば、異なる、しかも同じ変数を全く含んでいないようなSNPを使ったモデルになることもありうる。これはモデルに価値がないということではない。例えば、そのモデルはある独立した患者のグループの血圧を予測するのに非常に効果的で、したがって臨床現場において医学的にとても役立つということもあるであろう。しかし、得られた結果を過信しないよう注意を払わなければならない。そして、血圧を予測する際、多くのモデルがありうるが、そのうちの1つを見つけたにすぎず、独立したデータセットでさらに検証しなくてはならないことを強く認識しなければならない。

高次元の場合は誤差やモデルの当てはめに関する評価については特に注意が必要である。$p > n$である場合、残差が0になる無益なモデルを作ることは簡単である。したがって誤差平方和、p値、$R^2$などの統計量、訓練データでの当てはまりの程度を表す他の古典的な量は、高次元におけるモデルの当てはまりの良さの証拠として決して用いるべきではない。独立したテストデータを使った結果や、交差検証誤差についての報告をすることが重要である。例えば、独立したテストデータを使ってMSEや$R^2$を検証することは有効な方法であるが、訓練MSEは有効ではない。

## 実習2: リッジ回帰とLasso

``Hitter``データ（野球選手の前年の成績に関する統計データ）でリッジ回帰を使って野球選手の``Salary``を予測することを考える。まずは不完全データ（欠測値を含むデータ）を削除する。

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
# scale()を用いるため
from sklearn.preprocessing import scale
# mean_squared_error()を用いるため
from sklearn.metrics import mean_squared_error

In [2]:
# RのISLRパッケージからHittersデータをcsvファイルにエクスポートする：
# データの保存先に移動してlibrary(ISLR)→data(Hitters, package="ISLR")→write.csv(Hitters, "Hitters.csv")

df = pd.read_csv('Datasets/Hitters.csv', index_col=0).dropna()
df.index.name = 'Player'
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, -Alan Ashby to -Willie Wilson
Data columns (total 20 columns):
AtBat        263 non-null int64
Hits         263 non-null int64
HmRun        263 non-null int64
Runs         263 non-null int64
RBI          263 non-null int64
Walks        263 non-null int64
Years        263 non-null int64
CAtBat       263 non-null int64
CHits        263 non-null int64
CHmRun       263 non-null int64
CRuns        263 non-null int64
CRBI         263 non-null int64
CWalks       263 non-null int64
League       263 non-null object
Division     263 non-null object
PutOuts      263 non-null int64
Assists      263 non-null int64
Errors       263 non-null int64
Salary       263 non-null float64
NewLeague    263 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 43.1+ KB


In [3]:
df.head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
Player,,,,,,,,,,,,,,,,,,,,
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N


In [4]:
# pandasのデータのデータ型の確認
df[['League', 'Division', 'NewLeague']].dtypes

League       object
Division     object
NewLeague    object
dtype: object

In [5]:
# 質的変数を変換しておく
# get_dummysはarraylikeオブジェクトを受け取りdataframeを返す
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
dummies.info()
print(dummies.head(3))

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, -Alan Ashby to -Willie Wilson
Data columns (total 6 columns):
League_A       263 non-null uint8
League_N       263 non-null uint8
Division_E     263 non-null uint8
Division_W     263 non-null uint8
NewLeague_A    263 non-null uint8
NewLeague_N    263 non-null uint8
dtypes: uint8(6)
memory usage: 3.6+ KB
               League_A  League_N  Division_E  Division_W  NewLeague_A  \
Player                                                                   
-Alan Ashby           0         1           0           1            0   
-Alvin Davis          1         0           0           1            1   
-Andre Dawson         0         1           1           0            0   

               NewLeague_N  
Player                      
-Alan Ashby              1  
-Alvin Davis             0  
-Andre Dawson            1  


In [6]:
y = df.Salary
# 独立変数からなる行列をつくる
# dropは dropしたいlist-like Index or column labelsを渡し、dataframeを返す（axis=0なら行方向に、1なら列方向にdropする）
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
# concatはseriesやdataframeを受け取りseriesやdataframeを返す（axis=0なら行方向に、1なら列方向に結合する）
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X.info()
X.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 263 entries, -Alan Ashby to -Willie Wilson
Data columns (total 19 columns):
AtBat          263 non-null float64
Hits           263 non-null float64
HmRun          263 non-null float64
Runs           263 non-null float64
RBI            263 non-null float64
Walks          263 non-null float64
Years          263 non-null float64
CAtBat         263 non-null float64
CHits          263 non-null float64
CHmRun         263 non-null float64
CRuns          263 non-null float64
CRBI           263 non-null float64
CWalks         263 non-null float64
PutOuts        263 non-null float64
Assists        263 non-null float64
Errors         263 non-null float64
League_N       263 non-null uint8
Division_W     263 non-null uint8
NewLeague_N    263 non-null uint8
dtypes: float64(16), uint8(3)
memory usage: 35.7+ KB


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
Player,,,,,,,,,,,,,,,,,,,
-Alan Ashby,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
-Alvin Davis,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
-Andre Dawson,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1


In [7]:
# RではHitters=na.omit(Hitters)→x=model.matrix(Salary~.,Hitters)[,-1]→set.seed(1)→sample()→write.csv()と実行した
# model.matrix()は質的変数があれば自動でダミー変数に変換するため計画行列を作成するのに便利な関数である（-1は切片の列）
# astype('float64')はstandardScaler()またはscale()のため
X_train = pd.read_csv('Datasets/Hitters_X_train.csv', index_col=0).astype('float64')
y_train = pd.read_csv('Datasets/Hitters_y_train.csv', index_col=0)
X_test = pd.read_csv('Datasets/Hitters_X_test.csv', index_col=0).astype('float64')
y_test = pd.read_csv('Datasets/Hitters_y_test.csv', index_col=0)
X_train.head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,LeagueN,DivisionW,PutOuts,Assists,Errors,NewLeagueN
-Mike Heath,288.0,65.0,8.0,30.0,36.0,27.0,9.0,2815.0,698.0,55.0,315.0,325.0,189.0,1.0,0.0,259.0,30.0,10.0,0.0
-John Russell,315.0,76.0,13.0,35.0,60.0,25.0,3.0,630.0,151.0,24.0,68.0,94.0,55.0,1.0,0.0,498.0,39.0,13.0,1.0
-Pete Incaviglia,540.0,135.0,30.0,82.0,88.0,55.0,1.0,540.0,135.0,30.0,82.0,88.0,55.0,0.0,1.0,157.0,6.0,14.0,0.0


In [8]:
from sklearn.preprocessing import StandardScaler
# 標準化のための平均と標準偏差を計算する
scaler = StandardScaler().fit(X_train)

In [9]:
ridge = Ridge(alpha=1.0)
# 学習データを標準化し学習する
ridge.fit(scaler.transform(X_train), y_train)
# テストデータで予測する
pred = ridge.predict(scaler.transform(X_test))
mean_squared_error(y_test, pred)

142388.81067638902

In [10]:
# 回帰係数を表示する
# np.flatten()は配列を1次元化しコピーを返す
pd.Series(ridge.coef_.flatten(), index=X.columns)

AtBat         -149.111160
Hits           109.234457
HmRun           70.378046
Runs           -18.727737
RBI             -9.910319
Walks          101.198612
Years         -103.592487
CAtBat        -196.124234
CHits          309.735609
CHmRun          45.556842
CRuns            5.048450
CRBI           136.690867
CWalks          -1.672710
PutOuts         40.627245
Assists        -76.291624
Errors          48.936357
League_N        75.934646
Division_W     -24.408753
NewLeague_N    -12.922267
dtype: float64

In [11]:
# alpha=1.0のときの回帰係数のノルム
np.sqrt(np.sum(ridge.coef_**2))

481.79294332074926

In [12]:
# 正則化項の影響を強くする
ridge.set_params(alpha=10**10)
# 単に標準化を行うscale()を使って標準化し学習する
# テストデータも同じように標準化するならばStandardScalerを使ってもよい（https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler）
ridge.fit(scale(X_train), y_train)
pred = ridge.predict(scale(X_test))
mean_squared_error(y_test, pred)

224669.88997662338

In [13]:
# リッジ回帰で非常に大きなalphaの値を使うことは、切片のみのモデルを当てはめてテストデータの予測に訓練データの平均を使うことと等価
((y_train.mean()-y_test)**2).mean()

x    224669.906736
dtype: float64

In [14]:
# alpha=10**10のときの回帰係数
pd.Series(ridge.coef_.flatten(), index=X.columns)

AtBat          1.941828e-06
Hits           2.027949e-06
HmRun          2.271090e-06
Runs           2.277431e-06
RBI            2.453446e-06
Walks          2.656953e-06
Years          2.043934e-06
CAtBat         3.090458e-06
CHits          3.212319e-06
CHmRun         3.422507e-06
CRuns          3.301981e-06
CRBI           3.464895e-06
CWalks         3.230358e-06
PutOuts        1.277013e-07
Assists       -1.575202e-06
Errors         1.374508e-06
League_N       1.517945e-07
Division_W    -1.716398e-08
NewLeague_N   -3.930355e-08
dtype: float64

In [15]:
# alpha=10**10のときの回帰係数のノルム
# alpha=1.0のときよりも確かに小さくなっている
np.sqrt(np.sum(ridge.coef_**2))

1.0238323616801042e-05

In [91]:
alphas = 10**np.linspace(10,-2,100)
# cvを指定しなければLOOCVが行われる（https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html）
# cv=で指定するとiidパラメータのdeprication warningが出てきてしまう
# scoringではモデル評価の方法を指定する（https://scikit-learn.org/stable/modules/model_evaluation.html）
ridgecv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridgecv.fit(scale(X_train), y_train)

RidgeCV(alphas=array([1.00000e+10, 7.56463e+09, ..., 1.32194e-02, 1.00000e-02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring='neg_mean_squared_error', store_cv_values=False)

In [92]:
ridgecv.alpha_

75.64633275546291

In [93]:
ridge.set_params(alpha=ridgecv.alpha_)
ridge.fit(scale(X_train), y_train)
mean_squared_error(y_test, ridge.predict(scale(X_test)))

128848.7927451657

In [94]:
lasso = Lasso()
lassocv = LassoCV(alphas=alphas, cv=10, max_iter=10000)
# np.ravel()はnp.flatten()より高速だがインプレース（コピーをとらず、そのオブジェクトを直接変更する）なため必要ならflatten()を使う
lassocv.fit(scale(X_train), y_train.values.flatten())

LassoCV(alphas=array([1.00000e+10, 7.56463e+09, ..., 1.32194e-02, 1.00000e-02]),
    copy_X=True, cv=10, eps=0.001, fit_intercept=True, max_iter=10000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [95]:
lassocv.alpha_

10.72267222010321

In [96]:
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(scale(X_train), y_train)
mean_squared_error(y_test, lasso.predict(scale(X_test)))

130685.62080218211

In [97]:
# 係数のいくつかは正確に0に縮小推定されている
pd.Series(lasso.coef_, index=X.columns)

AtBat           0.000000
Hits            0.000000
HmRun          30.216650
Runs            0.000000
RBI             0.000000
Walks          86.710223
Years         -38.658642
CAtBat          0.000000
CHits          85.746895
CHmRun         56.857298
CRuns           0.000000
CRBI           97.368965
CWalks          0.000000
PutOuts        21.517098
Assists       -66.349728
Errors         31.962209
League_N       20.278556
Division_W     -0.000000
NewLeague_N     0.000000
dtype: float64

## Reference

- 落海浩，首藤信通．Rによる統計的学習入門．朝倉書店，2018，403p．
- https://github.com/JWarmenhoven/ISLR-python